**Auteur**: Voahangy Joan Aléonard<br>
**Dernière mise à jour** : 21 juillet 2021

# <font color=red>**Modélisation des systèmes de recommandation**</font>

**My Content** est une start-up (fictive) qui souhaite **proposer une application mobile de recommandation d’articles** à ses utilisateurs.

L'**objectif général** d'une méthode de recommandation est de ***prédire les valeurs manquantes*  d'une matrice d'utilité**, où chaque utilisateur est associé à une ligne et chaque article à une colonne.

| | A1 | A2 | A3 | A4 | ... |
|:--: | :--: | :--: | :--: | :--: | :--: |
| **U1** | 1 |  |  | 1 |  |
| **U2** |  |  | 1 | 1 |  |
| **U3** | 1 | 1 |  |  | 1 |
| **U4** |  | 1 | 1 | 1 | 1 |

Ce notebook a pour objectif de présenter les étapes de construction de 3 modèles de recommandation:
- Un modèle **non personnalisé** basé sur la popularité des articles ;
- Un modèle **basé sur le contenu** (*content-based*) ;
- Un modèle basé sur le **filtrage collaboratif** (*collaborative filtering*).

# <font color=green>**Chargement des données**</font>

---



In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import os

def load_data(src_path, click_path):
    '''
    Fonction utilisée pour charger les données du site "News Portal" de Globo.com

    Parameters:
        src_path : chemin vers le dossier contenant les metadonnées et la matrice embedding des articles
        click_path : chemin vers le dossier contenant les fichiers d'interactions des utilisateurs (1 fichier par heure)
    
    Returns:
        articles (dataframe) : les métadonnées des articles (id, catégorie, date de publication, nombre de mots)
        embeddings (array) : matrice de 250 vecteurs des mots contenus dans les articles
        clicks (dataframe)  : les interactions ou clicks des utilisateurs avec les articles
    '''
    # Load articles' metadata
    articles = pd.read_csv(src_path / 'articles_metadata.csv')
    # Drop useless feature
    articles.drop(columns=['publisher_id'], inplace=True)
    # Convert all data types to integer
    articles = articles.astype(np.int64)

    # Load articles' embedding
    embeddings = pd.read_pickle(src_path / 'articles_embeddings.pickle')
    # Change data type from float64 to float32
    embeddings = embeddings.astype(np.float32)

    # Load user interactions with articles
    clicks = pd.DataFrame().append([pd.read_csv(click_path / file) for file in sorted(os.listdir(click_path))],ignore_index=True)
    # Rename columns
    clicks.rename(columns={'click_article_id':'article_id'}, inplace=True)
    # Drop useless feature
    clicks.drop(columns=['click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type'], inplace=True)
    # Convert all data types to integer
    clicks = clicks.astype(np.int64)

    return articles, embeddings, clicks

In [2]:
# Set general configurations
src_path = Path('/content/drive/MyDrive/OC_IA/P09/data')
click_path = Path('/content/drive/MyDrive/OC_IA/P09/data/clicks')

# Call the function to load data
articles, embeddings, clicks = load_data(src_path, click_path)

# Display shape
print('Articles Dataframe shape: ', articles.shape)
print('Embedding Matrix shape: ', embeddings.shape)
print('Clicks Dataframe shape: ', clicks.shape)

Articles Dataframe shape:  (364047, 4)
Embedding Matrix shape:  (364047, 250)
Clicks Dataframe shape:  (2988181, 6)


In [3]:
# Display DF shape and 5 first rows
print('Articles Dataframe shape: ', articles.shape)
articles.head()

Articles Dataframe shape:  (364047, 4)


,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [4]:
# Display DF shape and 5 first rows
df_embeddings = pd.DataFrame(embeddings)
print('Embedding Matrix shape: ', embeddings.shape)
df_embeddings.head()

Embedding Matrix shape:  (364047, 250)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,0.428434,0.355056,0.874437,-0.528883,0.625487,0.268920,-0.822835,-0.703853,-0.625845,-0.152855,-0.666241,0.043295,0.178638,0.046890,0.594531,-0.183348,0.195107,-0.467640,-0.304807,0.353175,0.278188,0.538623,-0.371209,0.489898,-0.103833,0.119174,0.132437,-0.621084,-0.453311,0.346627,...,0.764381,0.275644,-0.692138,-0.393240,-0.325693,0.163377,-0.154952,-0.701639,0.711825,-0.839063,-0.459145,0.919913,0.621834,-0.640471,0.389934,0.764178,0.164851,-0.747023,0.411034,0.750901,0.860643,0.614755,0.549660,0.334437,-0.388059,-0.703730,-0.567420,0.006478,-0.206590,-0.385272,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,0.672840,-0.011180,-0.347506,-0.541346,0.458492,0.482316,0.713665,-0.474050,0.405787,0.671431,-0.480856,0.434779,0.468499,0.376424,0.368048,0.192545,0.095572,-0.542234,0.065547,0.363267,-0.162804,0.319617,0.469473,0.578466,-0.803637,-0.304545,0.635511,0.089978,-0.569946,0.102307,...,0.344321,-0.534050,0.728764,0.860110,-0.028166,0.491114,0.531556,0.545350,-0.241544,-0.091119,0.001625,0.950923,-0.367805,-0.215411,-0.131891,0.709081,0.612876,0.814885,0.278680,0.003690,-0.286773,-0.100936,-0.382605,-0.191886,-0.416420,-0.856593,0.537442,0.281693,-0.681829,0.666095,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,0.670484,-0.280388,-0.557285,-0.084145,0.027782,0.294074,0.362697,-0.368549,0.147960,-0.011751,0.030209,0.106317,0.628013,0.388849,0.615911,-0.445113,0.106028,0.137109,-0.095536,0.342532,0.592646,-0.261791,0.342123,0.704539,-0.433067,0.104154,0.785971,0.588640,-0.627687,-0.143294,...,-0.600840,-0.539323,0.418196,0.028715,0.135008,-0.313483,0.744361,-0.145113,-0.485484,-0.591555,0.030112,0.952736,-0.569376,-0.359680,0.561676,0.381671,-0.189300,0.420950,0.022337,-0.329559,0.232765,0.424368,-0.484598,0.394343,-0.329527,-0.852293,0.634992,-0.532673,-0.469396,0.714991,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,0.591640,-0.245205,-0.788418,-0.345293,-0.756407,0.141115,0.283783,-0.369959,0.374733,0.922633,0.172219,0.253582,0.552108,-0.454131,0.385160,-0.151101,0.598559,-0.614190,-0.258765,0.591907,0.618749,0.414598,0.698184,-0.114862,-0.069056,-0.420455,0.589368,0.496948,-0.449707,-0.747113,...,-0.137121,-0.245490,0.170559,0.568089,-0.252295,-0.455726,0.756765,0.626615,-0.005399,-0.472355,0.324625,0.954345,-0.840553,-0.800332,0.111637,-0.501806,0.115986,0.805840,-0.041481,0.320630,-0.450113,-0.378700,0.509616,0.142787,0.149670,-0.896181,0.234389,-0.189831,-0.597612,0.263807,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,0.698668,0.467503,-0.740664,-0.258062,-0.094110,0.256941,0.582071,-0.025041,0.339164,-0.348473,0.413076,0.159945,0.511137,0.608007,0.626225,-0.540690,0.623640,-0.019697,0.027607,-0.236370,0.311671,-0.559831,0.766381,0.412553,-0.663338,0.154626,0.708132,0.214954,-0.829963,-0.804490,...,0.234426,-0.689234,0.751153,0.647424,-0.033631,-0.578617,0.820915,0.370377,-0.528718,-0.536308,-0.088349,0.944439,-0.887679,-0.617357,0.254590,0.084341,0.286366,0.360714,0.369629,-0.452543,-0.292962,-0.530023,-0.237123,0.233008,0.002609,-0.835145,0.494137,0.029535,-0.619638,0.813445,0.2382

In [5]:
# Display DF shape and 5 first rows
print('Clicks Dataframe shape: ', clicks.shape)
clicks.head()

Clicks Dataframe shape:  (2988181, 6)


,user_id,session_id,session_start,session_size,article_id,click_timestamp
0,0,1506825423271737,1506825423000,2,157541,1506826828020
1,0,1506825423271737,1506825423000,2,68866,1506826858020
2,1,1506825426267738,1506825426000,2,235840,1506827017951
3,1,1506825426267738,1506825426000,2,96663,1506827047951
4,2,1506825435299739,1506825435000,2,119592,1506827090575


# <font color=green>**Rappel des données essentielles**</font>

---

In [6]:
print('Nombre total d\'utilisateurs: ', clicks.user_id.nunique())
print('Nombre total d\'articles: ', articles.article_id.nunique())
print('Nombre d\'articles consultés par les utilisateurs: ', clicks.article_id.nunique())
# print('Avg number of articles read by user for all sessions: ', round(clicks.groupby('user_id')['article_id'].nunique().mean(), 1))
# print('Avg number of user interaction per article for all sessions: ', round(clicks.groupby('article_id')['user_id'].nunique().mean(), 1))
# rint('Avg number of session per user: ', round(clicks.groupby('user_id')['session_id'].nunique().mean(), 1))

Nombre total d'utilisateurs:  322897
Nombre total d'articles:  364047
Nombre d'articles consultés par les utilisateurs:  46033


# <font color=green>**Recommandation non personnalisé: par popularité**</font>

---

Ce modèle ne recommande pas au sens propre du terme, c'est-à-dire de manière personnalisée.

Il **suggère à l'utilisateur les articles les plus populaires**, et si possible, ceux que l'utilisateur n'a pas encore "consommé".

Cette approche peut être efficace en cas de nouvel utilisateur, sans aucun historique d'interaction.

In [7]:
# Create popularity DF
def get_Popularity_Reco(clicks, n_reco=5):
    # Compute the most popular articles
    df_popularity = clicks.groupby(by=['article_id'])['click_timestamp'].count().sort_values(ascending=False).reset_index()
    df_popularity.rename(columns = {'user_id':'popularity'}, inplace=True)
    return df_popularity.article_id.head(n_reco).to_list()

In [8]:
# Call the function
get_Popularity_Reco(clicks, n_reco=5)

[160974, 272143, 336221, 234698, 123909]

# <font color=green>**Recommandation personnalisé type content-based**</font>

---

Le **Content-based Filtering** est un système de recommandation qui suggère des articles similaires à ceux déjà vus par l'utilisateur.

**Principe** : 
- A partir des descriptions des articles choisis ou notés par un utilisateur, un *profil* de l'utilisateur est construit ;
- Des articles dont les caractéristiques sont similaires au profil de l'utilisateur lui sont proposés.

Plus le nombre d'articles choisis ou notés par un utilisateur est faible, moins le profil obtenu et donc les recommandations faites sur cette base seront fiables.

Ce système se base donc sur **l'historique utilisateur et un accès à des *descriptions* des articles est indispensable**: si l'utilisateur A a lu 2 articles sur les chats, le système peut lui recommander d'autres articles sur les animaux de compagnie.

**Inconvénients** : 
- S'il est simple d'identifier des articles *de substitution*, il est clairement difficile d'extrapoler d'un domaine à un autre.

In [9]:
# Import libraries
import pandas as pd
import numpy as np
from time import time
from random import randint
from sklearn.metrics.pairwise import cosine_similarity

def get_ContentBased_Reco(userID, clicks, embeddings, n_reco=5):
    '''Return 5 recommended articles ID to user'''

    # Print targetted UserID
    print('Le n° d\'utilisateur est: ', userID)

    # Get the list of articles viewed by the user
    var = clicks.loc[clicks.user_id == userID]['article_id'].to_list()
    
    # Select the last element of the list (most recent one)
    value = var[-1]
    print('Le dernier article vu par l\'utilisateur est: ', value)
    
    # Delete already viewed articles except the selected one
    emb = embeddings
    for i in range (0, len(var)):
        if i != value:
            emb = np.delete(emb, [i], 0)
    
    # Delete selected article from the new matrix
    temp = np.delete(emb, [value], 0)

    # Get n_reco articles which are the most similar to the selected one
    distances = cosine_similarity([emb[value]], temp)[0]
    
    # Find the indexes, except the selected article
    ranked_ids = np.argsort(distances)[::-1][0:n_reco]
    ranked_similarities = np.sort(distances)[::-1][0:n_reco]
    print('Les articles recommandés sont: ')
    
    
    return ranked_ids.tolist()# , ranked_similarities.tolist()

In [10]:
# Call the function
start = time()

userID = 0
reco = get_ContentBased_Reco(userID, clicks, embeddings, n_reco=5)
print(reco)

print(f'Modèle exécuté en {round(time() - start, 2)}s')

Le n° d'utilisateur est:  0
Le dernier article vu par l'utilisateur est:  87205
Les articles recommandés sont: 
[102720, 100020, 102412, 102611, 86703]
Modèle exécuté en 1.69s


# <font color=green>**Recommandations personnalisé type collaborative filtering**</font>

---

Le **Collaborative Filtering** est un système de recommandation qui va tenter de prédire les préférences d'un utilisateur en se basant sur les préférences semblables d'autres utilisateurs.

Cette approche se base uniquement sur la **matrice d'utilités**, aucune connaissance intrinsèque des articles n'est nécessaire.

**Principe** :
- Proposer à un utilisateur des articles similaires à ceux qu'il a déjà choisi (ou bien noté) OU proposer à un utilisateur des articles choisis (ou bien notés) par les utilisateurs similaires.

Ce système se base donc **à la fois sur la similarité entre les articles et entre utilisateurs** pour faire ses recommendations: si un utilisateur A est similaire à l'utilisateur B, et que l'utilisateur B a lu l'article 1, le système peut recommander l'article 1 à l'utilisateur A même s'il n'a jamais regardé des articles similaires.

**Inconvénients** :

S'il n'y a pas d'étanchéité entre les domaines, en revanche, l'ignorance des caractéristiques des articles rend difficile la susbstitution d'articles manquants.

In [11]:
from IPython.display import clear_output

# Install a library
!pip install implicit

clear_output()

In [12]:
# Import libraries
import pandas as pd
import numpy as np
from time import time
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares as ALS
from implicit.lmf import LogisticMatrixFactorization as LMF

def get_CollabFilter_Reco(clicks, userID, model='ALS', n_reco=5):

    start = time()

    # Create interaction DF (count of interactions between users and articles)
    interactions = clicks.groupby(['user_id','article_id']).size().reset_index(name='count')
    # print('Interactions DF shape: ', interactions.shape)

    # csr = compressed sparse row (good format for math operations with row slicing)
    # Create sparse matrix of shape (number_items, number_user)
    csr_item_user = csr_matrix((interactions['count'].astype(float),
                                (interactions['article_id'],
                                 interactions['user_id'])))
    # print('CSR Shape (number_items, number_user): ', csr_item_user.shape)
    
    # Create sparse matrix of shape (number_user, number_items)
    csr_user_item = csr_matrix((interactions['count'].astype(float),
                                (interactions['user_id'],
                                 interactions['article_id'])))
    # print('CSR Shape (number_user, number_items): ', csr_user_item.shape)

    # Train the model on sparse matrix of shape (number_items, number_user)

    if model == 'ALS':
        # factors (int) : number of latent factors to compute
        # regularization (float) : regularization factor to use
        # iterations (int) : number of ALS iterations to use when fitting data
        # random_state (int) : seed the initial item and user factors (default:None)
        model = ALS(factors=128, regularization=0.01, iterations=30, random_state=42)
        # Calculate the confidence for each value in the data
        # low confidence level for non-read articles
        # high confidence level for read articles
        ALPHA_VAL = 40
        confidence = (csr_item_user * ALPHA_VAL).astype('double')
        model.fit(confidence)

    elif model == 'LMF':
        model = LMF(factors= 128, random_state=42)
        # model.approximate_similar_items = False
        model.fit(csr_item_user)

    # Recommend N best items from sparse matrix of shape (number_user, number_items)
    # Implicit built-in method
    # N (int) : number of results to return
    # filter_already_liked_items (bool) : if true, don't return items present in 
      # the training set that were rated/viewd by the specified user
    recommendations_list = []
    recommendations = model.recommend(userID, csr_user_item, N=n_reco, filter_already_liked_items=True)
    
    # Display i (items), not j (confidence score)
    for i, j in recommendations:
        recommendations_list.append(i)
    recommendations_list

    print(f'Completed in {round(time() - start, 2)}s')
    return recommendations_list

In [13]:
# Call the function
userID = 0
get_CollabFilter_Reco(clicks, userID=0, model='ALS', n_reco=5)


Completed in 12.12s


[160940, 284985, 158906, 337735, 272218]

In [14]:
# Call the function
userID = 0
get_CollabFilter_Reco(clicks, userID=0, model='LMF', n_reco=5)

100%|██████████| 30/30 [03:21<00:00,  6.73s/it]

Completed in 208.02s


[208155, 159359, 332901, 16346, 234390]

# <font color=green>**Synthèse et amélioration possible**</font>

---

Lorsqu'on possède à la fois des descriptions des articles et une matrice d'utilités, il est souhaitable de tirer profit de ces deux sources d'information pour améliorer les prédictions. 

Les systèmes de recommandations actuels emploient généralement des combinaisons d'approche, d'où la dénomination de **systèmes hybrides**.